In [1]:
!wget "https://data.sdss.org/sas/dr17/env/APOGEE_STARHORSE/APOGEE_DR17_EDR3_STARHORSE_v2.fits"

--2025-06-19 11:49:29--  https://data.sdss.org/sas/dr17/env/APOGEE_STARHORSE/APOGEE_DR17_EDR3_STARHORSE_v2.fits
Resolving data.sdss.org (data.sdss.org)... 155.101.19.133
Connecting to data.sdss.org (data.sdss.org)|155.101.19.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 289100160 (276M) [application/octet-stream]
Saving to: ‘APOGEE_DR17_EDR3_STARHORSE_v2.fits’

APOGEE_DR17_EDR3_ST 100%[===================>] 275.71M  5.64MB/s    in 45s     

2025-06-19 11:50:15 (6.12 MB/s) - ‘APOGEE_DR17_EDR3_STARHORSE_v2.fits’ saved [289100160/289100160]



In [1]:
from astropy.table import Table

data = Table.read("APOGEE_DR17_EDR3_STARHORSE_v2.fits")

/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float64'> type is zero.
  return self._float_to_str(self.smallest_subnormal)


In [ ]:
from mmoma.datasets.astropile import FastAstroPileLoader

dm = FastAstroPileLoader(
    "/mnt/home/polymathic/ceph/MultimodalUniverse/gaia_v2/", 
    dataset_name="dr3_xp", 
    include_healpix=[2068],
    num_workers=8,
)
dm.setup("")
loader = dm.train_dataloader()

/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:549: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  setattr(self, word, getattr(machar, word).flat[0])
/mnt/sw/nix/store/71ksmx7k6xy3v9ksfkv5mp5kxxp64pd6-python-3.10.13-view/lib/python3.10/site-packages/numpy/core/getlimits.py:89: UserWarning: The value of the smallest subnormal for <class 'numpy.float32'> type is zero.
  return self._float_to_str(self.smallest_subnormal)
/mnt/home/lparker/venv/mmoma/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:445: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  @autocast(enabled = False)
/mnt/home/lparker/venv/mmoma/lib/python3.10/site-packages/vector_quantize_pytorch/vector_quantize_pytorch.py:630: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use

Resolving data files:   0%|          | 0/3072 [00:00<?, ?it/s]

In [ ]:
import torch
import numpy as np
from tqdm import tqdm

t = Table.read("APOGEE_DR17_EDR3_STARHORSE_v2.fits")

starhorse_ids = np.array(t['EDR3_source_id']).astype(int)
targets = np.array(t[['teff50', 'logg50', 'met50']]).view("<f4").reshape(len(t), 3)

In [ ]:
saved_gaia_ids = []
saved_bp = []
saved_rp = []
saved_labels = []

for batch in tqdm(loader):
    gaia_ids = np.asarray(batch['object_id'], dtype=int)
    _, gaia_ix, starhorse_ix = np.intersect1d(gaia_ids, starhorse_ids, return_indices=True)

    saved_gaia_ids.append(gaia_ids[gaia_ix])
    saved_bp.append(batch['bp_coefficients'][gaia_ix])
    saved_rp.append(batch['rp_coefficients'][gaia_ix])
    saved_labels.append(targets[starhorse_ix])

saved_gaia_ids = torch.from_numpy(np.concatenate(saved_gaia_ids))
saved_bp = torch.from_numpy(np.concatenate(saved_bp))
saved_rp = torch.from_numpy(np.concatenate(saved_rp))
saved_labels = torch.from_numpy(np.concatenate(saved_labels))